In [2]:
from opyenxes.model.XLog import XLog
from opyenxes.data_in.XUniversalParser import XUniversalParser
from opyenxes.classification.XEventAttributeClassifier import XEventAttributeClassifier
import pandas as pd
import numpy as np

In [3]:
bpic_2011_path = "../logs/bpic2011.xes"

with open(bpic_2011_path) as bpic2011_file:
    bpic2011_rlog = XUniversalParser().parse(bpic2011_file)

Unknown extension: http://www.xes-standard.org/meta_time.xesext
Unknown extension: http://www.xes-standard.org/meta_life.xesext
Unknown extension: http://www.xes-standard.org/meta_org.xesext
Unknown extension: http://www.xes-standard.org/meta_concept.xesext
Unknown extension: http://www.xes-standard.org/meta_3TU.xesext
Unknown extension: http://www.xes-standard.org/meta_general.xesext


In [4]:
bpic2011_log = bpic2011_rlog[0] # the rest of this array is empty anyway as len(bpic2011_rlog) == 1
raw_trace = bpic2011_log[0]
raw_event = raw_trace[0]
raw_attributes = raw_event.get_attributes()

## Attributes of the first event of the first trace

In [5]:
for attribute in raw_attributes:
    attribute = raw_attributes[attribute]
    print(attribute.get_key(), "=", attribute.get_value())

org:group = Radiotherapy
Number of executions = 1
Specialism code = 61
concept:name = 1e consult poliklinisch
Producer code = SRTH
Section = Section 5
Activity code = 410100
time:timestamp = 2005-01-03 00:00:00+01:00
lifecycle:transition = complete


## All attributes of the first trace and their occurence along it

In [6]:
# collect all attributes
attribute_list = []

for event in raw_trace:
    event_attributes = event.get_attributes()
    
    for attribute in event_attributes:
        attribute_list.append(attribute)
        
attribute_list = set(attribute_list) # remove duplicates

event_indices = range(0,len(raw_trace))
attribute_labels = attribute_list
attribute_dict = {attr:[] for attr in attribute_list}

trace_df = pd.DataFrame(columns=attribute_labels, index=event_indices)

for event_idx,event in enumerate(raw_trace):
    event_attributes = event.get_attributes()
    for attribute in event_attributes:
        trace_df.iloc[event_idx][attribute] = event_attributes[attribute].get_value()

In [11]:
# Test prefixspan here
from prefixspan import PrefixSpan
import itertools

event_sequences = [[ ev.get_attributes()["concept:name"].get_value() for ev in trace ] for trace in bpic2011_log ]
combined_traces = list(itertools.chain.from_iterable(event_sequences))
events = sorted(list(set(combined_traces)))
event_to_int = dict((c, i) for i, c in enumerate(events))
int_to_event = dict((i,c) for i,c in enumerate(events))

translated_sequences = [ [event_to_int[ev] for ev in trace] for trace in event_sequences]

ps = PrefixSpan(translated_sequences)

In [27]:
def print_pattern(pt):
    for p in pt:
        print("Support: {0}%".format(p[0]/len(translated_sequences)))
        for n in p[1]:
            print("    > ", int_to_event[n])
        print()

In [30]:
ps_topkc = ps.topk(15, closed=True)
print("Top 15 closed prefixes")
print_pattern(ps_topkc)

ps_topk  = ps.topk(15, generator=True)
print("\nTop 15 prefixes")
print_pattern(ps_topk)
    
#print("\nTop 10 generator prefixes")
#for p in ps_topg:  print(p)

#print("\nLength 2 prefixes")
#for p in ps_freq2: print(p)
    
#print("\nLength 3 prefixes")
#for p in ps_freq3: print(p)

#ps_freq4 = ps.frequent(4, closed=True)
#print("\nLength 4 prefixes")
#for p in ps_freq4: print(p)

Top 15 closed prefixes
Support: 0.9711286089238845%
    >  administratief tarief       - eerste pol

Support: 0.8381452318460193%
    >  vervolgconsult poliklinisch

Support: 0.7340332458442694%
    >  administratief tarief       - eerste pol
    >  vervolgconsult poliklinisch

Support: 0.7034120734908137%
    >  aanname laboratoriumonderzoek
    >  ordertarief

Support: 0.6692913385826772%
    >  aanname laboratoriumonderzoek
    >  aanname laboratoriumonderzoek
    >  ordertarief

Support: 0.6649168853893264%
    >  vervolgconsult poliklinisch
    >  vervolgconsult poliklinisch

Support: 0.6377952755905512%
    >  aanname laboratoriumonderzoek
    >  hemoglobine foto-elektrisch
    >  ordertarief

Support: 0.6325459317585301%
    >  aanname laboratoriumonderzoek
    >  aanname laboratoriumonderzoek
    >  hemoglobine foto-elektrisch
    >  ordertarief

Support: 0.6325459317585301%
    >  aanname laboratoriumonderzoek
    >  ordertarief
    >  administratief tarief       - eerste pol


TypeError: 'int' object is not subscriptable